In [1]:
import pandas as pd
from IPython.display import display

# process web visits

the web_visits dataset has unneeded columns. the title and description columns are just more informative of the url. when specific url is visited, it always has the same endpoint.

In [2]:
train_path = r"..\data\train\web_visits.csv"
test_path = r"..\data\test\test_web_visits.csv"
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df=pd.concat([df_train, df_test], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521069 entries, 0 to 521068
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   member_id    521069 non-null  int64 
 1   url          521069 non-null  object
 2   title        521069 non-null  object
 3   description  521069 non-null  object
 4   timestamp    521069 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.9+ MB


In [4]:
df["timestamp"].min()

'01/07/2025 00:00'

In [2]:
df["url_category"] = df["url"].str.split("/").str[3]
df["url_category"].unique()

array(['chronic', 'tech', 'heart', 'gaming', 'stress', 'sleep',
       'wellness', 'aerobic', 'diabetes', 'strength', 'hypertension',
       'movies', 'cardio', 'nutrition', 'sports', 'fitness',
       'mindfulness', 'pets', 'cars', 'finance', 'travel', 'recipes',
       'weight'], dtype=object)

In [3]:
def check_consistency(df, col_1, col_2):
    # check whether each title has a single unique description
    desc_counts = df.groupby(col_1)[col_2].nunique().sort_values(ascending=False)

    total_col_1 = desc_counts.shape[0]
    inconsistent = desc_counts[desc_counts > 1]

    print(f"Total distinct {col_1}s: {total_col_1}")
    print(f"{col_1.capitalize()}s with >1 distinct {col_2}: {inconsistent.size}")

    if inconsistent.size == 0:
        print(f"Every {col_1} has a single (identical) {col_2}.")
    else:
        # show top titles with the most distinct descriptions
        display(inconsistent.head(20))
    sample_examples = (
        df[df[col_1].isin(inconsistent.index)]
        .groupby(col_1)[col_2]
        .unique()
        .apply(list)
        .reset_index()
        .rename(columns={col_2: f"{col_2}"})
    )
    display(sample_examples.sample(n=min(10, sample_examples.shape[0]), random_state=0))

In [4]:
check_consistency(df, col_1="title", col_2="description")
print("   -----------------------   ")
check_consistency(df, col_1="description", col_2="title")
print("   -----------------------   ")

Total distinct titles: 26
Titles with >1 distinct description: 0
Every title has a single (identical) description.


,title,description


   -----------------------   
Total distinct descriptions: 26
Descriptions with >1 distinct title: 0
Every description has a single (identical) title.


,description,title


   -----------------------   


we can see that title and description are connected to each other for each title always the same description and for each title the same description. 
we can drop the decription.

In [5]:
df.drop(columns=["description"], inplace=True)

In [6]:
check_consistency(df, col_1="url_category", col_2="url")
print("   -----------------------   ")


Total distinct url_categorys: 23
Url_categorys with >1 distinct url: 23


url_category
cars            3600
travel          3600
movies          3600
sports          3600
tech            3600
nutrition       3599
sleep           3596
heart           3595
gaming          3491
hypertension    3457
fitness         3449
aerobic         3448
cardio          3448
mindfulness     3442
diabetes        3440
stress          3439
wellness        3438
recipes         3436
strength        3430
weight          3427
Name: url, dtype: int64

,url_category,url
11,movies,"[https://portal.site/movies/821, https://porta..."
10,mindfulness,"[https://living.better/mindfulness/758, https:..."
21,weight,"[https://living.better/weight/102, https://liv..."
14,recipes,"[https://guide.wellness/recipes/159, https://g..."
20,travel,"[https://media.hub/travel/490, https://world.n..."
1,cardio,"[https://care.portal/cardio/181, https://guide..."
13,pets,"[https://portal.site/pets/371, https://example..."
22,wellness,"[https://guide.wellness/wellness/814, https://..."
16,sports,"[https://portal.site/sports/267, https://media..."
8,heart,"[https://health.wellco/heart/792, https://guid..."


   -----------------------   


In [7]:
check_consistency(df, col_1="title", col_2="url_category")
print("   -----------------------   ")
check_consistency(df, col_1="url_category", col_2="title")

Total distinct titles: 26
Titles with >1 distinct url_category: 0
Every title has a single (identical) url_category.


,title,url_category


   -----------------------   
Total distinct url_categorys: 23
Url_categorys with >1 distinct title: 3


url_category
sleep        2
nutrition    2
heart        2
Name: title, dtype: int64

,url_category,title
2,sleep,"[Restorative sleep tips, Sleep hygiene]"
1,nutrition,"[Mediterranean diet, Cholesterol friendly foods]"
0,heart,"[Hypertension basics, Cardiometabolic health]"


we see that we need to merge title and url_category to keep all the information about the category

In [8]:
df["category"]= df["url_category"] + "_" + df["title"]

In [9]:
df.drop(columns=["title", "url_category"], inplace=True)
display(df.head(10))

,member_id,url,timestamp,category
0,1,https://health.wellco/chronic/859,02/07/2025 22:38,chronic_Diabetes management
1,1,https://portal.site/tech/328,02/07/2025 11:30,tech_Gadget roundup
2,1,https://health.wellco/heart/792,14/07/2025 00:38,heart_Hypertension basics
3,2,https://example.com/gaming/674,07/07/2025 02:56,gaming_Game reviews
4,2,https://living.better/stress/325,02/07/2025 15:53,stress_Stress reduction
5,2,https://care.portal/sleep/928,02/07/2025 13:41,sleep_Restorative sleep tips
6,2,https://guide.wellness/wellness/814,01/07/2025 00:37,wellness_Healthy eating guide
7,2,https://care.portal/aerobic/384,07/07/2025 14:04,aerobic_Aerobic exercise
8,2,https://guide.wellness/heart/881,04/07/2025 16:29,heart_Hypertension basics
9,2,https://health.wellco/diabetes/194,01/07/2025 13:19,diabetes_HbA1c targets


when training the model we will try with and without the full url as input. maybe more feature spliting. 

In [10]:
df["domain"] = df["url"].str.split("/").str[2]
df["domain"].unique()

array(['health.wellco', 'portal.site', 'example.com', 'living.better',
       'care.portal', 'guide.wellness', 'world.news', 'media.hub'],
      dtype=object)

In [11]:
import sys, os

# Add the project root (adjust ".." as needed)
sys.path.append(os.path.abspath("../.."))  # go up one directory
from wellco_churn.src.process_datasets import get_web_feats


In [12]:
get_web_feats(train_path)

,member_id,total_visits,domain_care.portal,domain_example.com,domain_guide.wellness,domain_health.wellco,domain_living.better,domain_media.hub,domain_portal.site,domain_world.news,...,category_recipes_High-fiber meals,category_sleep_Restorative sleep tips,category_sleep_Sleep hygiene,category_sports_Match highlights,category_strength_Strength training basics,category_stress_Stress reduction,category_tech_Gadget roundup,category_travel_Top destinations,category_weight_Weight management,category_wellness_Healthy eating guide
0,1,3,0,0,0,2,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,2,9,3,1,2,2,1,0,0,0,...,0,1,0,0,1,1,0,0,0,1
2,3,52,3,7,10,6,4,5,11,6,...,1,3,1,4,0,0,3,3,0,0
3,4,15,1,3,2,1,3,1,3,1,...,0,1,1,0,1,1,0,1,0,0
4,5,11,2,1,2,0,2,1,3,0,...,0,1,1,1,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9970,9996,18,5,2,4,4,0,1,1,1,...,0,1,1,0,0,1,0,1,1,0
9971,9997,8,0,2,2,2,0,0,2,0,...,0,0,0,1,1,0,1,2,0,1
9972,9998,8,0,1,0,1,2,0,2,2,...,0,1,0,0,0,0,1,2,0,0
9973,9999,36,2,9,4,3,3,4,3,8,...,3,1,1,3,1,1,4,1,0,0


# process claims

In [3]:
train_path = r"..\data\train\claims.csv"
test_path = r"..\data\test\test_claims.csv"
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df = pd.concat([df_train, df_test], ignore_index=True)

In [6]:
df["icd_code"].nunique()

10

In [7]:
# show unique ICD codes and counts
unique_codes = df["icd_code"].unique()
print(f"Unique ICD codes (n={len(unique_codes)}):")
print(", ".join(sorted(unique_codes)))

# show top ICD codes by frequency
print("\nTop ICD codes by frequency:")
display(df["icd_code"].value_counts().head(30))

Unique ICD codes (n=10):
A09, B34.9, E11.9, H10.9, I10, J00, K21.9, M54.5, R51, Z71.3

Top ICD codes by frequency:


icd_code
Z71.3    18297
I10      18241
E11.9    18023
M54.5    10804
B34.9    10778
A09      10777
R51      10708
H10.9    10621
J00      10579
K21.9    10551
Name: count, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129379 entries, 0 to 129378
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   member_id       129379 non-null  int64 
 1   icd_code        129379 non-null  object
 2   diagnosis_date  129379 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.0+ MB


# app usage

Days since the last web visit or app use

In [5]:
train_path = r"..\data\train\app_usage.csv"
test_path = r"..\data\test\test_app_usage.csv"
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df = pd.concat([df_train, df_test], ignore_index=True)

In [12]:
# ensure timestamps are datetimes
df["timestamp"] = pd.to_datetime(df["timestamp"])

# treatment date (same as before)
treatment_date = pd.to_datetime("2025-07-15")

# get last event timestamp per member and compute days until treatment
last_by_member = (
    df.groupby("member_id")["timestamp"]
    .max()
    .reset_index()
    .rename(columns={"timestamp": "last_timestamp"})
)
last_by_member["treatment_date"] = treatment_date
last_by_member["days_since_last_app_use"] = (
    last_by_member["treatment_date"] - last_by_member["last_timestamp"]
).dt.days

last_by_member.head(20)

,member_id,last_timestamp,treatment_date,days_since_last_app_use
0,1,2025-07-14 15:28:58,2025-07-15,0
1,2,2025-07-14 13:55:31,2025-07-15,0
2,3,2025-07-14 18:15:54,2025-07-15,0
3,4,2025-07-13 22:17:45,2025-07-15,1
4,5,2025-07-13 22:06:17,2025-07-15,1
5,6,2025-07-14 23:14:58,2025-07-15,0
6,7,2025-07-14 07:01:48,2025-07-15,0
7,8,2025-07-14 02:15:27,2025-07-15,0
8,9,2025-07-14 16:32:24,2025-07-15,0
9,10,2025-07-14 08:59:19,2025-07-15,0


In [14]:
last_by_member["days_since_last_app_use"].min()

np.int64(0)